## 이름 : 오재영 

## 학번 : 2017250024

> # 목표

타이타닉 호에서 탑승했던 사람들의 정보를 바탕으로 생존자를 예측

> # 기본세팅

In [ ]:
# 모듈 임포트
import numpy as np
import pandas as pd
import os

# kaggle 제공 데이터셋 경로 불러오기
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [ ]:
# 데이터셋
train = pd.read_csv("/kaggle/input/titanic/train.csv")
test = pd.read_csv("/kaggle/input/titanic/test.csv")

데이터셋 확인

In [ ]:
train.head()

> # **Kaggle** 에서 제공되는 **Data Dictionary**

정리된 데이터 사전이다.

```
변수        정의	                                키

survival   생존 여부                                0 = No, 1 = Yes
pclass     등급(1등석, 2등석, 3등석)                 1 = 1st, 2 = 2nd, 3 = 3rd
sex        성별
Age        나이	
sibsp      타이타닉에 탑승한 형재,자매 / 배우자의 수	
parch      타이타닉에 탑승한 부모님 / 자녀의 수	
ticket     티켓 번호	
fare       여객 운임 (요금)	
cabin      객실 번호	
embarked   승선 항                                  C = Cherbourg, Q = Queenstown, S = Southampton
```

train 데이터 정보 확인

In [ ]:
print(train.shape)
train.info()

> # 데이터 시각화

데이터를 시각화하여 데이터 통계치 분포 확인

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

def bar_chart(feature):
    # 특성 변수에 따른 생존 승객을 카운트
    survived = train[train['Survived']==1][feature].value_counts()
    # 특성 변수에 따른 사망 승객을 카운트
    dead = train[train["Survived"]==0][feature].value_counts()
    # 생존 승객과, 사망 승객을 한눈에 보도록 데이터프레임으로 묶는다.
    df = pd.DataFrame([survived,dead])
    df.index = ['Survived', ' Dead']
    # 막대 그래프로 표현
    df.plot(kind='bar',stacked=True,figsize=(10,5))

In [ ]:
bar_chart('Sex') # 성별에 따른 생존 여부

막대 그래프를 통해서 보면 여자가 더 많이 살아남았고, 남자가 더 많이 죽었다.

In [ ]:
bar_chart('Pclass') # 등급에 따른 생존 여부

3등급 승객이 가장 많이 죽었다.

In [ ]:
bar_chart('Embarked') # 승선한 곳에 따른 생존 여부

Southampton 에서 승선한 승객이 가장 많고 가장 많이 죽었다.

하지만 이렇게 보면 정확한 비율을 모르기 때문에
범주형 데이터의 범주 별 생존율을 알고 싶다.

In [ ]:
# 항목 별 비율을 알려주는 원그래프 함수를 작성
def pie_chart(feature):
    
    feature_ratio = train[feature].value_counts(sort=False) # 특성에 속해있는 모든 범주들 카운트
    feature_size = feature_ratio.size
    feature_index = feature_ratio.index
    survived = train[train['Survived'] == 1][feature].value_counts() # 특성에 따른 생존 승객 카운트
    dead = train[train['Survived'] == 0][feature].value_counts() # 특성에 따른 사망 승객 카운트

    # 해당 항목의 전체 비율을 알려주는 원그래프
    plt.plot(aspect='auto') # 축 x와 y 비율을 자동으로 설정
    # 특성 전체를 범주 별로 쪼개서, 퍼센트 비율을 소수점 두자리까지 보여줌
    plt.pie(feature_ratio, labels=feature_index, autopct='%1.2f%%')
    plt.title(feature + '\'s ratio in total') # 그래프 제목 설정
    plt.show()
    
    # 해당 항목의 범주 별 생존율을 알려주는 원그래프
    for i, index in enumerate(feature_index):
        plt.subplot(1, feature_size, i + 1, aspect='equal') # 메인 그래프 밑에 조그맣게 볼 수 있도록 함
        plt.pie([survived[index], dead[index]], labels=['Survivied', 'Dead'], autopct='%1.2f%%')
        plt.title(str(index) + '\'s ratio')

    plt.show()

In [ ]:
pie_chart('Sex')

남성과 여성의 비율은 각각 64.76%, 35.24% 로 남성의 비율이 더 많지만,

생존율을 비교해 보았을 때 남성의 생존율은 18.89%, 여성의 생존율은 74.20% 로 여성의 생존율이 더 높다.



*1등석, 2등석, 3등석 간의 생존율 파악*

In [ ]:
pie_chart('Pclass')

전체 등급의 비율은 3등석 승객의 비율이 55.11%로 가장 높다.

생존율은 1등석 승객이 62.96%로 가장 높고, 3등석 승객이 24.24%로 가장 낮다.

In [ ]:
pie_chart('Embarked')

Southampton에서 승선한 승객의 비율이 72.44%로 가장 높고, Queenstown에서 승선한 승객의 비율은 8.66%로 가장 낮다.

Cherbourg에서 승선한 승객의 생존율이 55.36%로 가장 높고, Southampton에서 승선한 승객의 생존율이 33.70%로 가장 낮다.  

> # 데이터 전처리

* 이름 

`Name`에서 Mr, Miss 등 성별과 결혼 여부를 알 수 있는 타이틀을 추출하고 `Name` 항목은 삭제하려고 한다.

In [ ]:
# train과 test 세트를 합친 리스트 생성
train_test = [train, test]



In [ ]:
train_test

[891 rows x 12 columns]의 `train` 세트와
[418 rows x 11 columns]의 `test` 세트를 
리스트 형태로 합쳤다.

정규표현식을 이용하여 Mr, Mrs와 같은 타이틀을 추출한다.

In [ ]:
for dataset in train_test:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand = False)

`([A-Za-z]+)\.` 풀이

* 대문자 A-Z로 시작하고
* 두번째 문자부터 소문자이고, 소문자가 하나 이상인
* 끝이 . 으로 끝나는 문자열

을 추출하여 `Title` 특성을 생성한다.

특성이 잘 만들어졌는지 확인

In [ ]:
train['Title'].value_counts()

In [ ]:
test['Title'].value_counts()

알고리즘이 텍스트를 읽을 수 없기 때문에 숫자형 데이터로 변환 시켜줘야 한다.

* Mr = 0
* Miss = 1
* Mrs = 2
* Others = 3

이렇게 설정을 해줄 것이다.

3은 Mr, Miss, Mrs를 제외한 모든 것이다.

In [ ]:
title_conv = {"Mr": 0, "Miss": 1, "Mrs": 2,
            "Master": 3, "Dr": 3, "Rev": 3, "Col": 3, "Major": 3, "Mlle": 3,
            "Countess": 3, "Ms": 3, "Lady": 3, "Jonkheer": 3, "Don": 3,
            "Dona": 3, "Mme": 3, "Capt": 3, "Sir": 3}

for dataset in train_test:
    dataset['Title'] = dataset['Title'].map(title_conv)

잘 바뀌었는지 확인한다.

In [ ]:
train.head()

In [ ]:
test.head()

잘 바뀌었으므로 `Name` 특성 삭제

In [ ]:
train.drop('Name', axis=1, inplace=True)
test.drop('Name', axis=1, inplace=True)

In [ ]:
train.head()

In [ ]:
test.head()

`Title` 특성 시각화

In [ ]:
pie_chart('Title')

'Mr' 항목, 즉 남성의 비율은 58.02% 로 가장 많고,
생존율이 15.67% 로 굉장히 적다.

'Miss' 항목, 즉 미혼 여성의 비율은 20.43%,
69.78%의 생존율을 기록하고 있다.

'Mrs' 항목, 즉 기혼 여성의 비율은 14.03%,
생존율이 79.20%로 가장 높다. 

3가지로 나뉘지 않은 그 외 비율은 7.52%로 가장 낮고, 52.24%의 생존율을 기록하고 있다.


* 성별

마찬가지로 알고리즘이 알 수 있도록 숫자형 데이터로 변환.
* male = 0 (남)
* female = 1 (여)

In [ ]:
sex_conv =  {"male": 0, "female": 1}
for dataset in train_test:
    dataset['Sex'] = dataset['Sex'].map(sex_conv)

In [ ]:
pie_chart('Sex')

남성 64.76%, 남성의 생존율 18.89%
여성 35.24%, 여성의 생존율 74.20%

* 나이

나이에 누락된 값이 있기 때문에 누락된 값을 변환한다.

In [ ]:
train['Age'].value_counts(dropna=False)

In [ ]:
test['Age'].value_counts(dropna=False)

누락된 값을 평균값으로 변환하고

cut() 함수를 사용해서 구간을 나눌 것 이다.

Binnig 기법을 사용한다.

여러 종류의 데이터에 대해 범위를 지정해주거나 카테고리를 통해 이전보다 작은 수의 그룹으로 만드는 기법이다.

In [ ]:
for dataset in train_test:
    dataset['Age'].fillna(dataset['Age'].mean(), inplace=True) # 누락된 값 평균값으로 대체
    dataset['Age'] = dataset['Age'].astype(int) # 정수형 타입으로
    train['AgeBand'] = pd.cut(train['Age'], 5) # 5개의 구간으로 나눔
print (train[['AgeBand', 'Survived']].groupby(['AgeBand'], as_index=False).mean()) # 나눠진 5구간과 각각의 생존율

구간을 크게 5가지로 나누었다.

16세 이하 = 0

16세 초과 32세 이하 = 1

32세 초과 48세 이하 = 2

48세 초과 64세 이하 = 3

64세 초과 = 4


In [ ]:
for dataset in train_test:
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age'] = 4
    dataset['Age'] = dataset['Age'].astype(int)

In [ ]:
train.head()

`AgeBand`를 삭제한다.

In [ ]:
train.drop('AgeBand', axis=1, inplace=True)

`Cabin`과 `Ticket` 특성은 굳이 필요하지 않을 것으로 판단하여 삭제합니다.

In [ ]:
train.drop('Cabin', axis=1, inplace=True)
test.drop('Cabin', axis=1, inplace=True)
train.drop('Ticket', axis=1, inplace=True)
test.drop('Ticket', axis=1, inplace=True)

In [ ]:
train.head()

* 승선 항

`Embarked` 특성은 승객이 승선한 항에 대한 정보가 들어있다.

거주자 지역에 따라서도 생존율이 달라질 수 있으므로 어느 정도 유의미한 정보가 들어있다.

승선 항과 등급 간의 관계를 알고 싶다.

In [ ]:
Pclass1 = train[train['Pclass']==1]['Embarked'].value_counts()
Pclass2 = train[train['Pclass']==2]['Embarked'].value_counts()
Pclass3 = train[train['Pclass']==3]['Embarked'].value_counts()

# Embarked 특성에 따라 Pclass의 등급이 어떻게 나뉘는지 카운트.

In [ ]:
df = pd.DataFrame([Pclass1, Pclass2, Pclass3]) # 앞의 카운트를 데이터프레임 형식으로
df.index = ['1st', '2nd', '3rd'] # 항목의 이름
df.plot(kind='bar',stacked=True)
# 등급에 따른 승선 항을 위로 쌓는 방식의 막대그래프 생성.

Southampton 에서 탑승한 승객이 압도적으로 많고, Queenstown에서 탑승한 승객은 대부분 3등석이다.

Southampton 에서 탑승한 승객의 비율이 상대적으로 많이 높기 때문에 누락된 값을 Southampton을 뜻하는 `S`로 대체할 것이다.

In [ ]:
train['Embarked'].value_counts(dropna=False)

In [ ]:
for dataset in train_test:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

In [ ]:
train['Embarked'].value_counts(dropna=False)

그리고 숫자형 데이터로 변환 시킨다.

* S = 0
* C = 1
* Q = 2

In [ ]:
embarked_conv = {'S': 0, 'C': 1, 'Q': 2}
for dataset in train_test:
    dataset['Embarked'] = dataset['Embarked'].map(embarked_conv)

In [ ]:
train.head()

* 여객 운임 (요금)

`Fare` 는 요금을 뜻한다. 
구간을 정해서 등급을 나눌 것이다.

누락된 값이 하나 존재하는데, `Pclass`와 `Fare`가 어느 정도 연관성이 있는 것 같아 
`Fare` 데이터가 빠진 값의 `Pclass`를 가진 사람들의 평균 `Fare` 값을 넣는다.

In [ ]:
print(test[test["Fare"].isnull()]["Pclass"]) # 누락된 값의 Pclass 보여주기

152열의 `Fare` 값이 누락 되어 있고, `Pclass`는 3등급이다. 3등급 `Fare` 평균값을 구한다.

In [ ]:
print (train[['Pclass', 'Fare']].groupby(['Pclass'], as_index=False).mean())
# Pclass를 범주별로 묶어 범주별 Fare의 평균값을 출력

3등급의 평균 요금값은 약 13.7이다. 이 값을 누락된 값에 대체한다.

In [ ]:
for dataset in train_test:
    dataset['Fare'] = dataset['Fare'].fillna(13.7)

Age에서 한 과정을 그대로 한다.

구간은 크게 4구간으로 나눴다.

17 이하 = 0

17 초과 30 이하 = 1

30 초과 100 이하 = 2

100 초과 = 3

In [ ]:
for dataset in train_test:
    dataset.loc[ dataset['Fare'] <= 17, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 17) & (dataset['Fare'] <= 30), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 30) & (dataset['Fare'] <= 100), 'Fare'] = 2
    dataset.loc[ dataset['Fare'] > 100, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

In [ ]:
train.head()

* 가족

이제 마지막으로 

* `SibSp` : 형재,자매 / 배우자의 수, 
* `Parch` : 부모님 / 자녀의 수

이 두 특성의 관계를 하나로 묶어서 하나의 특성으로 만들어 볼 것이다.

! 가족 수 이기 때문에 자기 자신을 포함한다.

In [ ]:
train['Family'] = train['SibSp'] + train['Parch'] + 1
test['Family'] = test['SibSp'] + test['Parch'] + 1

# 1 = (자기 자신)

In [ ]:
train['Family'].max()

In [ ]:
test['Family'].max()

최대값이 11이고, 범위 구간을 좀 더 작게 정규화 시켜줄 것이다.

In [ ]:
family_conv = {1: 0, 2: 0.5, 3: 1, 4: 1.5,
              5: 2.0, 6: 2.5, 7: 3.0, 8: 3.5,
              9: 4, 10: 4.5, 11: 5}
for dataset in train_test:
    dataset['Family'] = dataset['Family'].map(family_conv)

In [ ]:
train.head()

마지막으로 합쳐진 특성 `SibSp`와 `Parch` 는 삭제 하도록 한다.

In [ ]:
features_drop = ['SibSp', 'Parch']
train = train.drop(features_drop, axis=1)
test = test.drop(features_drop, axis=1)
train = train.drop(['PassengerId'], axis=1) #인덱스 필요없음 
testnoid = test.drop(['PassengerId'], axis=1)

In [ ]:
train.head()

모든 전처리 과정이 끝났다.

In [ ]:
train.info()

> # 모델 구현

모델을 구현하고 교차 검증을 통해 정확도를 검증한다.

그 전에 `Survived` 특성을 타겟값으로 설정하기 위해 특성을 뗀 세트를 만들고 target 변수에 `Survived` 특성만 따로 떼어놓는다. 

In [ ]:
target = train['Survived']
train_data = train.drop('Survived', axis=1)

train_data.shape, train.shape

사용할 모델

* Logistic Regression : 로지스틱 회귀
* Support Vector Machine(SVM) : 서포트 벡터 머신
* k-Nearest Neighbor(kNN) : K-최근접 이웃
* Random Forest : 랜덤포레스트 회귀



필요한 scikit-learn 라이브러리를 불러오기

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

from sklearn.utils import shuffle

좀 더 정확한 학습을 위해 `shuffle` 을 이용해 데이터를 섞는다.

In [ ]:
train_data, target = shuffle(train_data, target, random_state = 5) # random_state : 난수 생성

하나의 함수만 만들어서 모든 모델을 학습하고 평가한다.

In [ ]:
def train_and_test(model):
    model.fit(train_data, target) # 훈련 세트를 타겟값에 대하여 학습
    prediction = model.predict(testnoid) # test 데이터 세트에 대해 타겟값 예측
    accuracy = round(model.score(train_data, target) * 100, 2) # 10번 시행시 평균 정확도
    print("정확도 : ", accuracy, "%")
    return prediction

In [ ]:
# 로지스틱 회귀
log_pred = train_and_test(LogisticRegression())
# SVM
svm_pred = train_and_test(SVC())
#kNN
knn_pred_4 = train_and_test(KNeighborsClassifier(n_neighbors = 4)) # 사용되는 이웃 수 4
# 랜덤포레스트
rf_pred = train_and_test(RandomForestClassifier(n_estimators=100)) # 사용되는 결정 트리 개수 100

랜덤 포레스트 회귀 모델로 학습 했을 때 정확도가 가장 높게 나왔다.

이 모델을 채택해서 submission 한다. (캐글에 제출할 csv 파일)

In [ ]:
submission = pd.DataFrame({
    "PassengerId": test["PassengerId"],
    "Survived": rf_pred
})

submission.to_csv('submission_rf.csv', index=False)